In [1]:
import pyarrow.csv
import requests
import pyarrow as pa
import pyarrow.fs
import io

In [2]:
#Q8: how many live DataNodes are in the cluster?
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 12364812597 (11.52 GB)
DFS Remaining: 12096536576 (11.27 GB)
DFS Used: 268276021 (255.85 MB)
DFS Used%: 2.17%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 80
	Missing blocks (with replication factor 1): 80
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.20.0.2:9866 (project-4-team-koslon-4-dn-1.project-4-team-koslon-4_default)
Hostname: c5ac4c6f32f3
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 268276021 (255.85 MB)
Non DFS Used: 13439463115 (12.52 GB)
DFS Remaining: 12096536576 (11.27 GB)
DFS Used%:

In [3]:
import time
time.sleep(30)

In [4]:
#Q9: how are the blocks of single.csv distributed across the DataNode containers?
url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS"
response = requests.get(url)
if response.status_code == 200:
    file_status = response.json()
    #print(file_status)
else:
    print(f"Error {response.status_code}: {response.text}")

block_locs = {}
key = 'hosts'

block_location_dict = file_status['BlockLocations']

block_location_list = block_location_dict['BlockLocation']

for block in block_location_list:
    host = block[key]
    if host == []:
        if 'lost' in block_locs:
            block_locs['lost'] += 1
        else:
            block_locs['lost'] = 1
    elif host[0] in block_locs:
        block_locs[host[0]] += 1
    else:
        block_locs[host[0]] = 1

block_locs

{'c5ac4c6f32f3': 87, 'lost': 80}

In [5]:
#Q10: how many times does the text "Single Family" appear in the remaining blocks of single.csv?
hdfs = pa.fs.HadoopFileSystem("boss",9000)
count = 0
tuples = []
for block in block_location_list:
    host = block[key]
    offset = 0
    length = 0
    if host == []:
        continue
    else:
        offset = block["offset"]
        length = block["length"]
        tuples.append((length, offset))
        #url = f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={offset}&length={length}&noredirect=true"
        #response = requests.get(url)
        #print(response.content)
with hdfs.open_input_file("/single.csv") as f:
        for t in tuples: 
            #print("HI")
            string = f.read_at(t[0],t[1])
            #print(string)
            text_string = string.decode('utf-8')
            if "Single Family" in text_string:
                count += 1

count


2023-10-27 17:12:04,727 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


87